# Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

**mlflow, version 1.26.0**

In [3]:
!mlflow --version

mlflow, version 1.26.0


# Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset. 

Download the data for January, February and March 2021 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/02-experiment-tracking/homework) to preprocess the data.

The script will

- load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
- fit a `DictVectorizer` on the training set (January 2021 data),
- save the preprocessed datasets and the model to disk

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* **4**
* 7


In [5]:
!python preprocess_data.py --raw_data_path ./input --dest_path ./output

In [8]:
import os 

len(os.listdir("./output"))

4

# Q3. Train a model with autolog

We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

* 19
* **17**
* 10
* 20

In [18]:
%%writefile train_modified.py
import argparse
import os
import pickle

import mlflow

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run(data_path):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))

    rf = RandomForestRegressor(max_depth=10, random_state=0)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_valid)

    rmse = mean_squared_error(y_valid, y_pred, squared=False)


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--data_path",
        default="./output",
        help="the location where the processed NYC taxi trip data was saved."
    )
    args = parser.parse_args()
    
    mlflow.set_tracking_uri("sqlite:///mlflow.db")
    mlflow.set_experiment("nyc-taxi-experiment")
    
    mlflow.autolog()

    with mlflow.start_run():
        run(args.data_path)

Overwriting train_modified.py


In [19]:
%run train_modified.py

2022/05/27 21:54:05 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/27 21:54:06 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/05/27 21:54:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/w10/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [20]:
!mlflow ui --backend-store-uri sqlite:///mlflow.db

[2022-05-27 21:54:44 +0000] [2979] [INFO] Starting gunicorn 20.1.0
[2022-05-27 21:54:44 +0000] [2979] [INFO] Listening at: http://127.0.0.1:5000 (2979)
[2022-05-27 21:54:44 +0000] [2979] [INFO] Using worker: sync
[2022-05-27 21:54:44 +0000] [2980] [INFO] Booting worker with pid: 2980
^C
[2022-05-27 21:56:04 +0000] [2979] [INFO] Handling signal: int
[2022-05-27 21:56:04 +0000] [2980] [INFO] Worker exiting (pid: 2980)


# Q4. Launch the tracking server locally

We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to

* launch the tracking server on your local machine
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* **`default-artifact-root`**
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`


In [ ]:
!mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts

# Q5. Tune the hyperparameters of the model

Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using `hyperopt`.

We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Important: don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization.
* the RMSE obtained on the validation set (February 2021 data).

What's the best validation RMSE that you got?

* 6.128
* **6.628**
* 7.128
* 7.628

In [21]:
%%writefile hpo_modified.py
import argparse
import os
import pickle

import mlflow
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")


def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run(data_path, num_trials):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))

    def objective(params):
        
        with mlflow.start_run():
            mlflow.log_params(params)
            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_valid)
            rmse = mean_squared_error(y_valid, y_pred, squared=False)
            mlflow.log_metric("rmse", rmse)

        return {'loss': rmse, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }

    rstate = np.random.default_rng(42)  # for reproducible results
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=Trials(),
        rstate=rstate
    )


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--data_path",
        default="./output",
        help="the location where the processed NYC taxi trip data was saved."
    )
    parser.add_argument(
        "--max_evals",
        default=50,
        help="the number of parameter evaluations for the optimizer to explore."
    )
    args = parser.parse_args()

    run(args.data_path, args.max_evals)


Writing hpo_modified.py


In [23]:
%run hpo_modified.py

2022/05/27 23:39:43 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.


  0%|                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

2022/05/27 23:40:01 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 4bb25e0257524bf4bce592c778d8092a. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'1.0\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fractio

  2%|▋                                 | 1/50 [00:17<14:40, 17.98s/trial, best loss: 6.658956269343007]

2022/05/27 23:40:05 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 0997c01d3cc84c8abc59a50fb728d216. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'1.0\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fractio

...

 96%|██████████████████████████████▋ | 48/50 [12:05<00:23, 11.78s/trial, best loss: 6.6284257482044735]

2022/05/27 23:52:02 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 2f4d99768aa64fee91efdf6a0e361b3f. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'1.0\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fractio

 98%|███████████████████████████████▎| 49/50 [12:19<00:12, 12.44s/trial, best loss: 6.6284257482044735]

2022/05/27 23:52:18 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID ea6fbff0ea20444cab2f63da3d090efe. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'1.0\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fractio

100%|████████████████████████████████| 50/50 [12:35<00:00, 15.10s/trial, best loss: 6.6284257482044735]


# Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE.
Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right model_uri in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 6.1
* **6.55**
* 7.93
* 15.1

In [46]:
%%writefile register_model_modified.py
import argparse
import os
import pickle

import mlflow
from hyperopt import hp, space_eval
from hyperopt.pyll import scope
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
EXPERIMENT_NAME = "random-forest-best-models"

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.sklearn.autolog()

SPACE = {
    'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
    'random_state': 42
}


def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def train_and_log_model(data_path, params):
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))
    X_test, y_test = load_pickle(os.path.join(data_path, "test.pkl"))

    with mlflow.start_run():
        params = space_eval(SPACE, params)
        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)

        # evaluate model on the validation and test sets
        valid_rmse = mean_squared_error(y_valid, rf.predict(X_valid), squared=False)
        mlflow.log_metric("valid_rmse", valid_rmse)
        test_rmse = mean_squared_error(y_test, rf.predict(X_test), squared=False)
        mlflow.log_metric("test_rmse", test_rmse)


def run(data_path, log_top):

    client = MlflowClient()

    # retrieve the top_n model runs and log the models to MLflow
    experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
    runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=log_top,
        order_by=["metrics.rmse ASC"]
    )
    for run in runs:
        train_and_log_model(data_path=data_path, params=run.data.params)

    # select the model with the lowest test RMSE
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
    best_run = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=log_top,
        order_by=["metrics.rmse ASC"])[0]

    # register the best model
    run_id = best_run.info.run_id
    model_uri = f"runs:/{run_id}/model"
    mlflow.register_model(model_uri = model_uri, name = "random-forest-best")


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--data_path",
        default="./output",
        help="the location where the processed NYC taxi trip data was saved."
    )
    parser.add_argument(
        "--top_n",
        default=5,
        type=int,
        help="the top 'top_n' models will be evaluated to decide which model to promote."
    )
    args = parser.parse_args()

    run(args.data_path, args.top_n)

Writing register_model_modified.py


In [47]:
%run register_model_modified.py

2022/05/28 00:19:02 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
Successfully registered model 'random-forest-best'.
2022/05/28 00:21:14 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-best, version 1
Created version '1' of model 'random-forest-best'.
